In [1]:
import grblas as gb

In [2]:
A = gb.Matrix.from_values(
  [0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4],
  [1, 2, 3, 4, 1, 3, 4, 1, 2, 4, 1, 2, 3, 5],
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  nrows=6, ncols=6
)
damping = 0.85
maxiter = 100
tolerance = 1e-6

In [3]:
# Ensure input has uniform edge weights
A = A.apply(gb.unary.one).new(dtype="FP32")

n = A.nrows
teleport = (1 - damping) / n
rdiff = 1  # first iteration is always done

# r = 1 / n
t = gb.Vector.new(gb.dtypes.FP32, n)
r = gb.Vector.new(gb.dtypes.FP32, n)
w = gb.Vector.new(gb.dtypes.FP32, n)
r[:] << 1.0 / n

# precompute d (row degree)
d = A.reduce_rows().new(dtype="FP32")
# prescale with damping factor, so it isn't done each iteration
d << d.apply(gb.binary.truediv, right=damping)

# --------------------------------------------------------------------------
# pagerank iterations
# --------------------------------------------------------------------------
for i in range(maxiter):
    if rdiff <= tolerance:
        break

    # swap t and r ; now t is the old score
    r, t = t, r

    # w = t ./ d
    w << gb.binary.truediv(t & d)

    # r = teleport
    r[:] << teleport

    # r += A'*w
    r(gb.binary.plus) << gb.semiring.plus_second(A.T @ w)

    # t -= r
    t(gb.binary.minus)[:] << r

    # t = abs (t)
    t << gb.unary.abs(t)

    # rdiff = sum (t)
    rdiff = t.reduce(gb.monoid.plus).value

if i == maxiter:
    print(f"Failed to converge in {maxiter} iterations")
else:
    print(f"Converged in {i} iterations")

Converged in 49 iterations


In [4]:
r

"v_0"          nvals  size  dtype  format
grblas.Vector      6     6   FP32    full
-----------------------------------------
      0        1         2         3         4         5
  0.025  0.15902  0.142461  0.142461  0.150784  0.057042